In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def load_stuff(fbase='results/2022-03-16_SLN_letters/noReparam_yesFlip_newInit_max'):
    all_params = []
    all_indices = []
    for letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
        data = np.load('{:}_{:}.npy'.format(fbase, letter), allow_pickle=True)
        # for trajectory in data:
        #     sol, history = trajectory
        #     strokes_params = sol['params']
        for trajectory in data:
            all_params += trajectory[0]['params']
            all_indices += trajectory[0]['stroke_indices'].values()
    all_params = np.array(all_params)
    all_indices = np.array(all_indices)
    print(all_params.shape, all_indices.shape)
    return all_params, all_indices

In [ ]:
def realign_t0(all_params, all_indices):
    t0s, ds, th1s, th2s, sigmas, mus = all_params.T
    DT = 1. / 120
    t_stroke_starts = all_indices[:, 0] * DT
    t0s = t0s - t_stroke_starts
    datas = {
        '$t_0$': t0s,
        '$D$': ds,
        '$\\theta_1$': th1s,
        '$\\theta_2$': th2s,
        '$\\sigma$': sigmas,
        '$\\mu$': mus
    }
    return datas

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(10, 5))
plt.subplots_adjust(hspace=0.3)

both_datas = {}
for artist in ['max', 'jules']:
    both_datas[artist] = realign_t0(*load_stuff(
        fbase='results/2022-03-16_SLN_letters/noReparam_yesFlip_newInit_{:}'.format(artist)))
mins = [min(*a) for a in zip(*[[np.min(data) for data in datas.values()] for datas in both_datas.values()])]
maxs = [max(*a) for a in zip(*[[np.max(data) for data in datas.values()] for datas in both_datas.values()])]
for artist, datas in both_datas.items():
    for ax, (label, data), lb, ub in zip(axes.flatten(), datas.items(), mins, maxs):
        ax.hist(data, bins=np.linspace(lb, ub, 30), label=artist, alpha=0.5)
        ax.set_title(label)
axes[0][1].legend()
fig.savefig('results/2022-03-16_SLN_letters/statistics.svg')